In [5]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np

model_path = os.getcwd() + '/tf_model/'
model_file = model_path + 'stock.model'

path = '/Users/liyangyang/Downloads/datafountain/taocan/'

In [4]:
data = pd.read_csv(path + 'train_all.csv')
data.loc[data['current_service'] == 90063345, 'current_service'] = 0
data.loc[data['current_service'] == 89950166, 'current_service'] = 1
data.loc[data['current_service'] == 89950167, 'current_service'] = 2
data.loc[data['current_service'] == 99999828, 'current_service'] = 3
data.loc[data['current_service'] == 90109916, 'current_service'] = 4
data.loc[data['current_service'] == 89950168, 'current_service'] = 5
data.loc[data['current_service'] == 99999827, 'current_service'] = 6
data.loc[data['current_service'] == 99999826, 'current_service'] = 7
data.loc[data['current_service'] == 90155946, 'current_service'] = 8
data.loc[data['current_service'] == 99999830, 'current_service'] = 9
data.loc[data['current_service'] == 99999825, 'current_service'] = 10

data.loc[data['age'] == '\\N', 'age'] = 0
data['age'] = data['age'].astype('int64')
data.loc[data['age'] < 20, 'age'] = 0
data.loc[(data['age'] >= 20) & (data['age'] < 30), 'age'] = 1
data.loc[(data['age'] >= 30) & (data['age'] < 40), 'age'] = 2
data.loc[(data['age'] >= 40) & (data['age'] < 50), 'age'] = 3
data.loc[data['age'] >= 50, 'age'] = 4

data.loc[data['gender'] == '\\N', 'gender'] = 0
data['gender'] = data['gender'].astype('int64')

data.loc[data['2_total_fee'] == '\\N', '2_total_fee'] = 0.0
data.loc[data['3_total_fee'] == '\\N', '3_total_fee'] = 0.0
data['2_total_fee'] = data['2_total_fee'].astype('float64')
data['3_total_fee'] = data['3_total_fee'].astype('float64')
data.loc[data['1_total_fee'] > 500.0, '1_total_fee'] = 500.0
data.loc[data['2_total_fee'] > 500.0, '2_total_fee'] = 500.0
data.loc[data['3_total_fee'] > 500.0, '3_total_fee'] = 500.0
data.loc[data['4_total_fee'] > 500.0, '4_total_fee'] = 500.0

data['total_fee'] = 0
data.loc[data['1_total_fee'] < .0, 'total_fee'] = 1
data.loc[data['2_total_fee'] < .0, 'total_fee'] = 1
data.loc[data['3_total_fee'] < .0, 'total_fee'] = 1
data.loc[data['4_total_fee'] < .0, 'total_fee'] = 1
data.loc[data['1_total_fee'] > 499.0, 'total_fee'] = 2
data.loc[data['2_total_fee'] > 499.0, 'total_fee'] = 2
data.loc[data['3_total_fee'] > 499.0, 'total_fee'] = 2
data.loc[data['4_total_fee'] > 499.0, 'total_fee'] = 2

data['month_traffic_0'] = 0
data.loc[(data['month_traffic'] > 0) & (data['month_traffic'] < 1024), 'month_traffic_0'] = 1
data.loc[data['month_traffic'] == 1024.0, 'month_traffic_0'] = 2
data.loc[data['month_traffic'] > 1024, 'month_traffic_0'] = 3

data.loc[data['online_time'] > 140, 'online_time'] = 140

data['pay_ave'] = data['pay_num'] / data['pay_times']
data.loc[data['pay_times'] > 10, 'pay_times'] = 10

data['my_traffic'] = data['last_month_traffic'].apply(lambda x: parse_traffic(x))

data = data.drop(['local_trafffic_month'], axis=1)
data = data.drop(['last_month_traffic'], axis=1)
data = data.drop(['month_traffic'], axis=1)

data.loc[data['local_caller_time'] == 0.0, 'local_caller_time'] = 0
data.loc[(data['local_caller_time'] > 0) & (data['local_caller_time'] < 10), 'local_caller_time'] = 1
data.loc[(data['local_caller_time'] >= 10) & (data['local_caller_time'] < 100), 'local_caller_time'] = 2
data.loc[data['local_caller_time'] >= 100, 'local_caller_time'] = 3

data.loc[data['service1_caller_time'] == 0.0, 'service1_caller_time'] = 0
data.loc[(data['service1_caller_time'] > 0) & (data['service1_caller_time'] < 10), 'service1_caller_time'] = 1
data.loc[(data['service1_caller_time'] >= 10) & (data['service1_caller_time'] < 100), 'service1_caller_time'] = 2
data.loc[data['service1_caller_time'] >= 100, 'service1_caller_time'] = 3

data.loc[data['service2_caller_time'] == 0.0, 'service2_caller_time'] = 0
data.loc[(data['service2_caller_time'] > 0) & (data['service2_caller_time'] < 10), 'service2_caller_time'] = 1
data.loc[(data['service2_caller_time'] >= 10) & (data['service2_caller_time'] < 100), 'service2_caller_time'] = 2
data.loc[data['service2_caller_time'] >= 100, 'service2_caller_time'] = 3

data['complaint_num'] = 0
data.loc[data['former_complaint_num'] > 0, 'complaint_num'] = 1

data['complaint_fee'] = 0
data.loc[data['former_complaint_fee'] > 0, 'complaint_fee'] = 1

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,5,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def parse_traffic(x):
    m = x / 1024.0
    if m == 0.0:
        return 0
    elif m < 1.0:
        return 0.5
    elif m == 1.0:
        return 1
    elif m < 2.0:
        return 1.5
    elif m == 2.0:
        return 2
    elif m < 3.0:
        return 2.5
    elif m == 3.0:
        return 3
    elif m < 4.0:
        return 3.5
    elif m == 4.0:
        return 4
    else:
        return 5

In [6]:
data.columns

Index(['service_type', 'is_mix_service', 'online_time', '1_total_fee',
       '2_total_fee', '3_total_fee', '4_total_fee', 'many_over_bill',
       'contract_type', 'contract_time', 'is_promise_low_consume',
       'net_service', 'pay_times', 'pay_num', 'local_caller_time',
       'service1_caller_time', 'service2_caller_time', 'gender', 'age',
       'complaint_level', 'former_complaint_num', 'former_complaint_fee',
       'current_service', 'user_id', 'total_fee', 'month_traffic_0', 'pay_ave',
       'my_traffic', 'complaint_num', 'complaint_fee'],
      dtype='object')

In [45]:
print(data['2_total_fee'].value_counts(sort=True))
print(max(data['2_total_fee']))
print(min(data['2_total_fee']))

76.00     19342
56.00     15382
36.00     12909
106.00    10971
136.00     6877
500.00     6289
166.00     4442
196.00     3895
86.00      3586
36.10      2890
50.00      2826
80.00      2821
76.10      2811
296.00     2811
96.00      2798
46.00      2601
69.00      2588
56.10      2572
0.00       2301
100.00     2149
126.00     1957
19.00      1945
140.00     1908
66.00      1841
95.00      1833
106.10     1745
120.00     1722
116.00     1705
81.00      1658
36.20      1579
          ...  
378.91        1
192.77        1
401.79        1
71.23         1
349.72        1
346.28        1
178.33        1
492.55        1
5.18          1
434.99        1
5.07          1
493.70        1
317.87        1
235.29        1
250.88        1
434.74        1
492.05        1
318.38        1
178.58        1
378.06        1
100.01        1
401.96        1
69.82         1
317.37        1
318.63        1
41.51         1
434.98        1
199.73        1
257.68        1
257.97        1
Name: 2_total_fee, Lengt

-0.1


In [43]:
data.loc[data['1_total_fee'] < .0, '1_total_fee'] = -0.1
data.loc[data['2_total_fee'] < .0, '2_total_fee'] = -0.1
data.loc[data['3_total_fee'] < .0, '3_total_fee'] = -0.1
data.loc[data['4_total_fee'] < .0, '4_total_fee'] = -0.1

In [6]:
import pandas as pd
test = pd.read_csv(path + 'republish_test.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 26 columns):
service_type              200000 non-null int64
is_mix_service            200000 non-null int64
online_time               200000 non-null int64
1_total_fee               200000 non-null float64
2_total_fee               200000 non-null object
3_total_fee               200000 non-null object
4_total_fee               200000 non-null float64
month_traffic             200000 non-null float64
many_over_bill            200000 non-null int64
contract_type             200000 non-null int64
contract_time             200000 non-null int64
is_promise_low_consume    200000 non-null int64
net_service               200000 non-null int64
pay_times                 200000 non-null int64
pay_num                   200000 non-null float64
last_month_traffic        200000 non-null float64
local_trafffic_month      200000 non-null float64
local_caller_time         200000 non-null float64
service1_

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
